In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings  # updated import
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

load_dotenv()
import pandas as pd

books = pd.read_csv(r"C:\Users\MSI\Desktop\Book Sphere\data\books_cleaned.csv")
print(books.head())

with open("tagged_description.txt", "w", encoding="utf-8") as f:
    for line in books["tagged_description"].tolist():
        f.write(line + "\n")

from langchain_community.document_loaders import TextLoader
from langchain.schema import Document

# Load document
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

# Manually split by newline (like chunk_size=0)
documents = []
for doc in raw_documents:
    lines = doc.page_content.split("\n")
    for line in lines:
        if line.strip():  # skip empty lines
            documents.append(Document(page_content=line))

print(f"Total chunks created: {len(documents)}")

documents[0]
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Example usage with Chroma
texts = ["This is a book about AI.", "This is a novel about love."]
db = Chroma.from_texts(texts, embeddings, persist_directory="db_books")

# Test search
query = "romantic stories"
results = db.similarity_search(query, k=1)
print(results)



!pip install -U ipywidgets

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Example usage with Chroma
texts = ["This is a book about AI.", "This is a novel about love."]
db = Chroma.from_texts(texts, embeddings, persist_directory="db_books")

# Test search
query = "romantic stories"
results = db.similarity_search(query, k=10)
print(results)

query = "romantic stories"
results = db.similarity_search(query, k=1)  # or k=10

# The first document
first_doc = results[0]

print(first_doc.page_content)

isbn_str = first_doc.page_content.split("\n")[0].strip()

try:
    isbn_num = int(isbn_str)
except ValueError:
    print(f"Cannot convert ISBN to int: {isbn_str}")

from langchain.schema import Document

# Create documents with ISBNs in metadata
documents = []
for idx, row in books.iterrows():
    documents.append(
        Document(
            page_content=row['tagged_description'],
            metadata={"isbn13": row['isbn13']}
        )
    )

# Build Chroma DB from these documents
db = Chroma.from_documents(documents, embeddings, persist_directory="db_books")

# Example search
query = "romantic stories"
results = db.similarity_search(query, k=1)

# Get ISBN from metadata
first_doc = results[0]
isbn_num = first_doc.metadata["isbn13"]
print("Matched ISBN:", isbn_num)

# Get book info from DataFrame
matched_books = books[books["isbn13"] == isbn_num]
print(matched_books)

# Example: search query
query = "books for babies about animals"
docs = db.similarity_search(query, k=10)

# Suppose the ISBN is the first number in the document content
doc_content = docs[0].page_content.strip()

# Remove extra spaces and split if needed
isbn_num = ''.join(filter(str.isdigit, doc_content))  # keep only digits
isbn_num = int(isbn_num)

matched_books = books[books["isbn13"] == isbn_num]

print(matched_books)

doc_content = docs[0].page_content.strip()
isbn_str = doc_content.split()[0]  # take first word if ISBN is at start
isbn_num = int(''.join(filter(str.isdigit, isbn_str)))  # keep only digits

matched_books = books[books["isbn13"] == isbn_num]
print(matched_books)


# Function to retrieve semantic recommendations
def retrieve_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    # Step 1: Semantic search in Chroma DB
    recs = db_books.similarity_search(query, k=50)  # search top 50 similar docs

    # Step 2: Extract ISBNs from the search results
    books_list = []
    for doc in recs[8:]:  # skip first 8 if needed
        try:
            isbn_str = doc.page_content.strip().split()[0]
            isbn_num = int(''.join(filter(str.isdigit, isbn_str)))
            books_list.append(isbn_num)
        except:
            continue  # skip if conversion fails

    # Step 3: Return matching books from CSV
    return books[books["isbn13"].isin(books_list)].head(top_k)


recommendations = retrieve_semantic_recommendations("A book to teach children about animals")
print(recommendations)

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Assuming you already have your documents split into 'documents'
# or you can use your texts column from the CSV
texts = books["tagged_description"].tolist()

# Build/load Chroma DB
db_books = Chroma.from_texts(texts, embeddings, persist_directory="db_books")

# Optional: persist the database (Chroma auto-persists in newer versions)
db_books.persist()

recommendations = retrieve_semantic_recommendations("A book to teach children about animals")
print(recommendations)

import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


def retrieve_semantic_recommendations(query: str, db: Chroma, books_df: pd.DataFrame, top_k: int = 10) -> pd.DataFrame:
    """
    Perform semantic search on a book vector database and return top matching books.

    Parameters:
        query (str): User query string.
        db (Chroma): Chroma vector database containing book embeddings.
        books_df (pd.DataFrame): Original book dataset with ISBNs and metadata.
        top_k (int): Number of recommendations to return.

    Returns:
        pd.DataFrame: Top-k recommended books with full metadata.
    """
    # Step 1: Semantic search
    recs = db.similarity_search(query, k=50)  # Search top 50 similar documents

    # Step 2: Extract ISBNs safely
    isbn_list = []
    for doc in recs:
        text = doc.page_content.strip()
        # Attempt to extract numeric ISBN from the start of the text
        parts = text.split()
        if parts:
            isbn_str = ''.join(filter(str.isdigit, parts[0]))
            if isbn_str.isdigit():
                isbn_list.append(int(isbn_str))

    # Step 3: Filter books by ISBN and return top_k
    recommended_books = books_df[books_df["isbn13"].isin(isbn_list)].head(top_k)

    return recommended_books


# Make sure db_books is your Chroma vector store and books is your DataFrame
query = "A book to teach children about animals"
recommendations = retrieve_semantic_recommendations(query, db_books, books, top_k=10)
print(recommendations)

query = "Stories about friendship and adventure"
recommendations = retrieve_semantic_recommendations(query, db_books, books, top_k=5)
print(recommendations)


          isbn13      isbn10                title  \
0  9780002005883  0002005883               Gilead   
1  9780002261982  0002261987         Spider's Web   
2  9780006178736  0006178731       Rage of angels   
3  9780006280897  0006280897       The Four Loves   
4  9780006280934  0006280935  The Problem of Pain   

                           authors                     categories  \
0               Marilynne Robinson                        Fiction   
1  Charles Osborne;Agatha Christie  Detective and mystery stories   
2                   Sidney Sheldon                        Fiction   
3              Clive Staples Lewis                 Christian life   
4              Clive Staples Lewis                 Christian life   

                                           thumbnail  \
0  http://books.google.com/books/content?id=KQZCP...   
1  http://books.google.com/books/content?id=gA5GP...   
2  http://books.google.com/books/content?id=FKo2T...   
3  http://books.google.com/books/content?i


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)

   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]

[Document(metadata={}, page_content='This is a novel about love.'), Document(metadata={}, page_content='This is a novel about love.'), Document(metadata={}, page_content='This is a book about AI.'), Document(metadata={}, page_content='This is a book about AI.')]
This is a novel about love.
Cannot convert ISBN to int: This is a novel about love.
